In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import rcParams

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
# Bengali Handwritten Character Recognition using Ekush Dataset
# Complete Implementation in Jupyter Notebook

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Display PyTorch version and GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

# Step 1: Define paths and parameters
# You'll need to update these paths according to your dataset location
BASE_PATH = r'C:\Users\KIIT\Desktop\minor\dataset\dataset'  # Update this path
METADATA_PATH = os.path.join(BASE_PATH, r"C:\Users\KIIT\Desktop\minor\metaData_img.csv")  # Update if filename is different
IMG_SIZE = 28  # We'll resize all images to 28x28
BATCH_SIZE = 28
EPOCHS = 15
NUM_CLASSES = 122  # 122 Bengali character classes (0-121)

# Step 2: Load metadata
# The metadata file maps folder names to actual Bengali characters
metadata = pd.read_csv(METADATA_PATH)
print("Metadata information:")
print(metadata.head())

# Create a mapping from class index to Bengali character name
class_to_char = dict(zip(metadata['Folder Name'].astype(int), metadata['Char Name']))
print(f"Sample mapping - Class 0: {class_to_char[0]}")

In [ ]:
# Step 3: PyTorch Dataset for loading and preprocessing images
from PIL import Image
from torch.utils.data import Dataset
import os
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Constants (define these as per your dataset)
IMG_SIZE = 64  # Example image size, adjust as needed
NUM_CLASSES = 10  # Example number of classes, adjust as needed

# Mapping from class index to character (define as per your dataset)
class_to_char = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J'}

# BengaliCharacterDataset class
class BengaliCharacterDataset(Dataset):
    def __init__(self, base_path, class_to_char, img_size=IMG_SIZE, transform=None):
        self.base_path = base_path
        self.class_to_char = class_to_char
        self.img_size = img_size
        self.transform = transform
        self.samples = []
        self.labels = []
        self._load_samples()

    def _load_samples(self):
        for class_idx in range(NUM_CLASSES):
            class_dir = os.path.join(self.base_path, str(class_idx))
            if not os.path.exists(class_dir):
                print(f"Warning: Directory for class {class_idx} not found at {class_dir}")
                continue
            img_files = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
            for img_file in img_files:
                img_path = os.path.join(class_dir, img_file)
                self.samples.append(img_path)
                self.labels.append(class_idx)
        print(f"Loaded {len(self.samples)} images.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path = self.samples[idx]
        label = self.labels[idx]
        try:
            img = Image.open(img_path).convert('L')  # Grayscale
            if self.transform:
                img = self.transform(img)
            else:
                img = transforms.ToTensor()(img)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            img = torch.zeros((1, self.img_size, self.img_size))
        return img, label

# Example transform
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

# Example usage:
# dataset = BengaliCharacterDataset(base_path='path_to_your_data', class_to_char=class_to_char, transform=transform)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Step 4: Load and prepare data using PyTorch Dataset and DataLoader

dataset = BengaliCharacterDataset(BASE_PATH, class_to_char, img_size=IMG_SIZE, transform=transform)

# Split indices for train, val, test
indices = np.arange(len(dataset))
train_idx, temp_idx, _, temp_labels = train_test_split(
    indices, dataset.labels, test_size=0.3, random_state=42, stratify=dataset.labels
)
val_idx, test_idx = train_test_split(
    temp_idx, test_size=0.5, random_state=42, stratify=temp_labels
)

# Subset Datasets
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)
test_dataset = Subset(dataset, test_idx)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

In [ ]:
# Step 5: Exploratory Data Analysis
# Visualize some sample images from the PyTorch Dataset
plt.figure(figsize=(12, 8))
for i in range(9):
    img, label = dataset[i]
    plt.subplot(3, 3, i+1)
    plt.imshow(img.squeeze().numpy(), cmap='gray')
    plt.title(f"Class: {label} ({class_to_char.get(label, 'Unknown')})")
    plt.axis('off')
plt.tight_layout()
plt.savefig('sample_bengali_characters.png')
plt.show()

# Check class distribution
plt.figure(figsize=(12, 6))
sns.countplot(x=dataset.labels)
plt.title('Distribution of Classes')
plt.xlabel('Class Index')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('class_distribution.png')
plt.show()

In [ ]:
# Step 6: Dataset splitting already handled above for PyTorch
# Labels are integer class indices (no one-hot encoding needed for PyTorch CrossEntropyLoss)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

In [ ]:
# Step 7: Data Augmentation using torchvision transforms
aug_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.9, 1.1)),
    transforms.ToTensor(),
])

# Visualize data augmentation examples
plt.figure(figsize=(12, 8))
for i in range(9):
    img, label = dataset[i]
    aug_img = aug_transform(transforms.ToPILImage()(img))
    plt.subplot(3, 3, i+1)
    plt.imshow(aug_img.squeeze().numpy(), cmap='gray')
    plt.title(f"Augmented Sample")
    plt.axis('off')
plt.tight_layout()
plt.savefig('augmented_samples.png')
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary

# Step 8: Define the CNN Model in PyTorch
class BengaliCNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(BengaliCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.4),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Create the model
model = BengaliCNN(num_classes=NUM_CLASSES)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Print model summary
def print_model_summary(model, input_size=(1, IMG_SIZE, IMG_SIZE)):
    summary(model, input_size)

try:
    print_model_summary(model)
except Exception as e:
    print(model)

In [ ]:
import torch.optim as optim
import torch.nn as nn

# Step 9: Define optimizer, loss function, and learning rate scheduler for PyTorch
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, min_lr=1e-5, verbose=True)

# Early stopping utility
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.verbose = verbose
        self.best_state = None

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            self.best_state = model.state_dict()
            if self.verbose:
                print(f"Validation loss improved to {val_loss:.4f}. Saving model.")
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# Step 10: Train the model (PyTorch training loop)
early_stopping = EarlyStopping(patience=5, verbose=True)
best_val_acc = 0.0
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_loss = running_loss / total
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
    val_loss = val_loss / val_total
    val_acc = val_correct / val_total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    scheduler.step(val_loss)
    early_stopping(val_loss, model)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_bengali_char_model.pth')
        print("Best model saved.")
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        model.load_state_dict(early_stopping.best_state)
        break

# Save training history for plotting
history = {
    'train_loss': train_losses,
    'val_loss': val_losses,
    'train_acc': train_accuracies,
    'val_acc': val_accuracies
}

In [ ]:
# Step 11: Visualize training history
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='Training Accuracy')
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

In [ ]:
# Step 12: Evaluate on test set (PyTorch)
model.eval()
test_loss = 0.0
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
test_loss = test_loss / test_total
test_acc = test_correct / test_total
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
# Step 13: Make predictions on test set (PyTorch)
model.eval()
y_pred = []
y_true = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.numpy())
y_pred = np.array(y_pred)
y_true = np.array(y_true)

In [ ]:
# Step 14: Classification report
print("Classification Report:")
class_report = classification_report(y_true, y_pred)
print(class_report)

# Save the classification report to file
with open('classification_report.txt', 'w') as f:
    f.write(class_report)

In [ ]:
# Step 15: Confusion Matrix (for a subset of classes)
# We'll visualize a subset due to the large number of classes
plt.figure(figsize=(12, 10))
subset_size = 20  # Visualize first 20 classes
subset_indices = np.where((y_true < subset_size) & (y_pred < subset_size))[0]

cm_subset = confusion_matrix(
    y_true[subset_indices],
    y_pred[subset_indices],
    labels=range(subset_size)
)

# Normalize confusion matrix
cm_normalized = cm_subset.astype('float') / cm_subset.sum(axis=1)[:, np.newaxis]

# Create heatmap
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=[class_to_char.get(i, str(i)) for i in range(subset_size)],
    yticklabels=[class_to_char.get(i, str(i)) for i in range(subset_size)]
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (First 20 Classes)')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png')
plt.show()

In [ ]:
# Step 16: Visualize some model predictions

def visualize_predictions(X_data, y_true, y_pred, num_samples=10):
    """Visualize model predictions alongside true labels"""
    indices = np.random.choice(len(X_data), num_samples, replace=False)

    plt.figure(figsize=(15, 6))
    for i, idx in enumerate(indices):
        plt.subplot(2, 5, i+1)
        img = X_data[idx]
        if isinstance(img, torch.Tensor):
            img = img.squeeze().numpy()
        plt.imshow(img, cmap='gray')

        true_label = class_to_char.get(y_true[idx], str(y_true[idx]))
        pred_label = class_to_char.get(y_pred[idx], str(y_pred[idx]))

        title = f"True: {true_label}\nPred: {pred_label}"
        color = "green" if y_true[idx] == y_pred[idx] else "red"

        plt.title(title, color=color)
        plt.axis('off')

    plt.tight_layout()
    plt.savefig('prediction_samples.png')
    plt.show()

# Visualize predictions
visualize_predictions([dataset[i][0] for i in test_idx], y_true, y_pred)

In [ ]:
# Step 17: Save the model for future use
# Save PyTorch model
model_path = r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pth"
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")

# Step 18: Create a simple prediction function for new images using PyTorch
def predict_bengali_character(img_path, model, class_to_char, device, img_size=IMG_SIZE):
    """
    Predict the Bengali character from a given image file using PyTorch.

    Args:
        img_path: Path to the image file
        model: Trained PyTorch model
        class_to_char: Mapping from class index to Bengali character
        device: torch.device
        img_size: Size to resize image

    Returns:
        predicted_class: The predicted class index
        confidence: Confidence score of the prediction
        character: Bengali character (if mapping available)
    """
    from PIL import Image
    model.eval()
    img = Image.open(img_path).convert('L')
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        confidence, predicted_class = torch.max(probs, 1)
        predicted_class = predicted_class.item()
        confidence = confidence.item()
    character = class_to_char.get(predicted_class, "Unknown")
    plt.figure(figsize=(4, 4))
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {character} (Class {predicted_class})\nConfidence: {confidence:.4f}")
    plt.axis('off')
    plt.show()
    return predicted_class, confidence, character

# Example usage (uncomment and update path when needed)
# model = BengaliCNN(num_classes=NUM_CLASSES)
# model.load_state_dict(torch.load(model_path, map_location=device))
# model.to(device)
# test_img_path = 'path/to/test/image.jpg'
# predicted_class, confidence, character = predict_bengali_character(test_img_path, model, class_to_char, device)
# print(f"Predicted character: {character} (Class {predicted_class}) with {confidence:.2%} confidence")

In [ ]:
plt.rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Noto Sans Bengali', 'Nirmala UI', 'Bangla Sangam MN', 'Lohit Bengali']

# Function to display images with Bengali text
def plot_bengali_predictions(images, true_labels, pred_labels, bengali_chars_dict, rows=2, cols=5, figsize=(15, 8)):
    """
    Plot images with Bengali labels
    
    Parameters:
    images: array of images to display
    true_labels: true class labels (numeric)
    pred_labels: predicted class labels (numeric)
    bengali_chars_dict: dictionary mapping class numbers to Bengali characters
    """
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten()
    
    for i, ax in enumerate(axes):
        if i < len(images):
            ax.imshow(images[i], cmap='gray')
            ax.axis('off')
            
            true_char = bengali_chars_dict.get(true_labels[i], str(true_labels[i]))
            pred_char = bengali_chars_dict.get(pred_labels[i], str(pred_labels[i]))
            
            # Set title with both correct and predicted labels
            ax.set_title(f'True: {true_char}\nPred: {pred_char}', 
                        color='green' if true_labels[i] == pred_labels[i] else 'red',
                        fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\minor\Screenshot 2025-04-09 054801.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\minor\Test_Img_05.jpg")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Pictures\Screenshots\Screenshot 2025-04-09 144032.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Downloads\img4.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.nn.functional.softmax(output, dim=1)[0] * 100

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence[predicted_index].item():.2f}%)")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\79.jpg")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pth")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\img8.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\img7.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pth")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\img9.png")  # Replace with actual path

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image

# Load your trained model
model = torch.jit.load(r"C:\Users\KIIT\Desktop\minor\ekkush_recognition_model.pt")
model.eval()

# Load the class index to Bengali character mapping from CSV
metadata_path = r"C:\Users\KIIT\Desktop\minor\metaData_img.csv"  # Replace with the actual path
metadata_df = pd.read_csv(metadata_path)

# Create the class_to_char dictionary from the DataFrame
class_to_char = dict(zip(metadata_df['Folder Name'], metadata_df['Char Name']))

# Define the image transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Predict function for a single image (PyTorch version)
def predict_image(img_path):
    # Load and preprocess image
    img = Image.open(img_path)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Predict class
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_index = torch.max(output, 1)
        confidence = torch.max(torch.nn.functional.softmax(output, dim=1))

    # Get corresponding Bengali character
    predicted_char = class_to_char.get(predicted_index.item(), "Unknown")

    # Display result
    plt.imshow(img, cmap='gray')
    plt.title(f"Predicted: {predicted_char} (Class {predicted_index.item()}, {confidence.item():.2f})")
    plt.axis('off')
    plt.show()

# Example usage:
predict_image(r"C:\Users\KIIT\Desktop\img11.png")  # Replace with actual path